# Setup

In [1]:
from neo4j import GraphDatabase
import json
import pandas as pd

with open('config.json') as f:
	config = json.load(f)

	uri = config['uri']
	username = config['username']
	password = config['password']

driver = GraphDatabase.driver(uri, auth=(username, password))


# Import Data

In [2]:
# Leggi il dataset
print("Caricamento del dataset...")
df = pd.read_csv('netflix.csv')
print(f"Dataset caricato con {len(df)} righe.")

# Preprocessing dei dati
print("Pulizia e preparazione dei dati...")
df['director'] = df['director'].fillna('')
df['cast'] = df['cast'].fillna('')
df['country'] = df['country'].fillna('')
df['date_added'] = df['date_added'].fillna('')
df['genres'] = df['genres'].fillna('')
df['rating'] = df['rating'].fillna('TV-MA')
df['duration'] = df['duration'].fillna('0 min')
print("Dati puliti e pronti per essere caricati nel database.")

# Funzione per creare nodi e relazioni
def create_nodes_and_relationships(tx, query, parameters=None):
    tx.run(query, parameters)

with driver.session() as session:
    # Creazione dei nodi per film e serie TV
    print("Creazione dei nodi per film e serie TV...")
    for idx, row in df.iterrows():
        if row['type'] == 'Movie':
            query = """
            CREATE (m:Movie {
                id: $id,
                title: $title,
                date_added: $date_added,
                release_year: $release_year,
                rating: $rating,
                duration: $duration,
                description: $description
            })
            """
        else:  # TV Show
            query = """
            CREATE (s:SerieTV {
                id: $id,
                title: $title,
                date_added: $date_added,
                release_year: $release_year,
                rating: $rating,
                seasons: $duration,
                description: $description
            })
            """
        session.execute_write(create_nodes_and_relationships, query, {
            'id': row['show_id'],
            'title': row['title'],
            'date_added': row['date_added'],
            'release_year': int(row['release_year']),
            'rating': row['rating'],
            'duration': row['duration'].replace(' min', '').replace(' Seasons', '').replace(' Season', ''),
            'description': row['description']
        })
        if idx % 100 == 0:
            print(f"Creati {idx+1} nodi di film/serie TV...")

    # Creazione dei nodi per le persone (registi e cast)
    print("Creazione dei nodi per registi e attori...")
    people = set()
    for _, row in df.iterrows():
        if row['director']:
            people.add(row['director'])
        cast_members = row['cast'].split(', ')
        people.update(cast_members)

    for idx, person in enumerate(people):
        if person:  # Salta stringhe vuote
            query = "CREATE (:Person {name: $name})"
            session.execute_write(create_nodes_and_relationships, query, {'name': person})
        if idx % 100 == 0:
            print(f"Creati {idx+1} nodi di persone...")

    # Collegare film e serie TV ai registi e al cast
    print("Creazione delle relazioni tra film/serie TV e registi/attori...")
    for idx, row in df.iterrows():
        if row['director']:
            query = """
            MATCH (p:Person {name: $name}), (m {id: $id})
            CREATE (p)-[:DIRECTED]->(m)
            """
            session.execute_write(create_nodes_and_relationships, query, {'name': row['director'], 'id': row['show_id']})

        cast_members = row['cast'].split(', ')
        for actor in cast_members:
            if actor:
                query = """
                MATCH (p:Person {name: $name}), (m {id: $id})
                CREATE (p)-[:ACTED_IN]->(m)
                """
                session.execute_write(create_nodes_and_relationships, query, {'name': actor, 'id': row['show_id']})
        if idx % 100 == 0:
            print(f"Create relazioni per {idx+1} film/serie TV...")

    # Creazione dei nodi per le categorie e collegamento con film e serie TV
    print("Creazione dei nodi per le categorie...")
    categories = set()
    for _, row in df.iterrows():
        genres = row['genres'].split(', ')
        categories.update(genres)

    for idx, category in enumerate(categories):
        if category:
            query = "CREATE (:Categoria {name: $name})"
            session.execute_write(create_nodes_and_relationships, query, {'name': category})
        if idx % 50 == 0:
            print(f"Creati {idx+1} nodi di categorie...")

    print("Collegamento dei film/serie TV alle categorie...")
    for idx, row in df.iterrows():
        genres = row['genres'].split(', ')
        for genre in genres:
            if genre:
                query = """
                MATCH (c:Categoria {name: $name}), (m {id: $id})
                CREATE (m)-[:IN_CATEGORY]->(c)
                """
                session.execute_write(create_nodes_and_relationships, query, {'name': genre, 'id': row['show_id']})
        if idx % 100 == 0:
            print(f"Collegate categorie per {idx+1} film/serie TV...")

    # Creazione dei nodi per i paesi e collegamento con film e serie TV
    print("Creazione dei nodi per i paesi...")
    countries = set(df['country'].unique())
    for idx, country in enumerate(countries):
        if country:
            query = "CREATE (:Country {name: $name})"
            session.execute_write(create_nodes_and_relationships, query, {'name': country})
        if idx % 20 == 0:
            print(f"Creati {idx+1} nodi di paesi...")

    print("Collegamento dei film/serie TV ai paesi...")
    for idx, row in df.iterrows():
        if row['country']:
            query = """
            MATCH (c:Country {name: $name}), (m {id: $id})
            CREATE (m)-[:WHERE]->(c)
            """
            session.execute_write(create_nodes_and_relationships, query, {'name': row['country'], 'id': row['show_id']})
        if idx % 100 == 0:
            print(f"Collegati paesi per {idx+1} film/serie TV...")

print("Tutti i dati sono stati caricati con successo nel database Neo4j!")
driver.close()


Caricamento del dataset...
Dataset caricato con 8807 righe.
Pulizia e preparazione dei dati...
Dati puliti e pronti per essere caricati nel database.
Creazione dei nodi per film e serie TV...
Creati 1 nodi di film/serie TV...
Creati 101 nodi di film/serie TV...
Creati 201 nodi di film/serie TV...
Creati 301 nodi di film/serie TV...
Creati 401 nodi di film/serie TV...
Creati 501 nodi di film/serie TV...
Creati 601 nodi di film/serie TV...
Creati 701 nodi di film/serie TV...
Creati 801 nodi di film/serie TV...
Creati 901 nodi di film/serie TV...
Creati 1001 nodi di film/serie TV...
Creati 1101 nodi di film/serie TV...
Creati 1201 nodi di film/serie TV...
Creati 1301 nodi di film/serie TV...
Creati 1401 nodi di film/serie TV...
Creati 1501 nodi di film/serie TV...
Creati 1601 nodi di film/serie TV...
Creati 1701 nodi di film/serie TV...
Creati 1801 nodi di film/serie TV...
Creati 1901 nodi di film/serie TV...
Creati 2001 nodi di film/serie TV...
Creati 2101 nodi di film/serie TV...
Creati